In [2]:
import pandas as pd
import numpy as np

In [1]:
import os

for dirname, _, filenames in os.walk('/home/daftvm/workspace/mast1_ai/ydays/real_estate/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/home/daftvm/workspace/mast1_ai/ydays/real_estate/real_estate.csv
/home/daftvm/workspace/mast1_ai/ydays/real_estate/real_estate.ipynb
